# Showcase YOLO

YOLOv8 is a state-of-the-art computer vision model built by Ultralytics. The YOLOv8 model contains out-of-the-box support for object detection, classification, and segmentation tasks, accessible through a Python package as well as a command line interface.

This example showcases one of YOLOv8 default models on a detection application.

## Setup

In [ ]:
!pip -q install ultralytics # Ignore if not using Colab

## Import Libraries

In [ ]:
from ultralytics import YOLO
import os
from urllib.request import urlretrieve
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import random

## User Inputs

In [ ]:
# Datasets folder
# Folder where all the datasets will be saved
datasets_folder = "datasets"

# Image URL to download and apply detection
# If getting error, use http instead of https, or use different image
image_url = "https://github.com/EmanuelCastanho/solar_panels_detection/blob/main/figs/test-example.png?raw=true"

## Code

In [ ]:
# Load a pretrained YOLOv8 model
# YOLOv8 Extra Large is the most accurate, but slowest among the five YOLO models
# The models are trained with COCO dataset, which include 80 classes
model = YOLO("models/yolov8x.pt")

In [ ]:
# Create new folder (if does not exist) to store the downloaded dataset
if not os.path.exists(datasets_folder):
  os.mkdir(datasets_folder)

In [ ]:
# Download image for detection
image_path = os.path.join(datasets_folder, "example-image.png")
urlretrieve(image_url, image_path)

In [ ]:
# Open and show image
image_bgr = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)

In [ ]:
# Run detection
results = model(source=image_bgr, show=False, conf=0.2, save=True)

In [ ]:
# Extract bounding boxes
boxes = results[0].boxes.xyxy.tolist()
classes = results[0].boxes.cls.tolist()
names = results[0].names
confidences = results[0].boxes.conf.tolist()

unique_classes = set(classes)
rand_color = ["#"+''.join([random.choice("0123456789ABCDEF") for j in range(6)]) for i in range(len(unique_classes))]
class_colors = {}
i=0
for u_clas in unique_classes:
    class_colors[u_clas] = rand_color[i]
    i += 1

bbox_info = {}
bbox_num = 1
for box, clas, conf in zip(boxes, classes, confidences):
    # Box: x1, y1, x2, y2
    bbox_key = "bbox_" + str(bbox_num)
    name = names[int(clas)]
    bbox_info[bbox_key] = [name, class_colors[int(clas)], round(conf,2), box]
    bbox_num += 1

In [ ]:
# Plot results
fig, ax = plt.subplots()
ax.imshow(image_rgb)
for bbox in bbox_info.keys():
    color = bbox_info[bbox][1]
    xy = (bbox_info[bbox][-1][0], bbox_info[bbox][-1][1])
    width = bbox_info[bbox][-1][2] - bbox_info[bbox][-1][0]
    height = bbox_info[bbox][-1][-1] - bbox_info[bbox][-1][1]
    label_text = bbox_info[bbox][0] + ": " + str(bbox_info[bbox][2])
    draw_bbox = patches.Rectangle(xy, width, height, linewidth=2, edgecolor=color, facecolor="none")
    ax.add_patch(draw_bbox)
    ax.text(xy[0], xy[1], s=label_text, fontsize=10, color="white", backgroundcolor=color)
plt.title("Detection")
plt.show()